In [1]:
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

from carla.data.causal_model import CausalModel


Using TensorFlow backend.


[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
scm = CausalModel("sanity-3-lin")
dataset = scm.generate_dataset(10000)

display(dataset.df)

Finished Loading
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix
Remove Prefix
End Prefix


,label,x1,x2,x3
0,1.0,-1.311620,2.522145,1.913979
1,0.0,0.506218,-2.099092,0.242393
2,1.0,1.647384,-1.812545,-0.518771
3,0.0,1.486129,-1.960024,-1.089874
4,0.0,2.675338,-2.767358,-2.408550
...,...,...,...,...
9995,0.0,0.254435,-1.489899,-0.985486
9996,1.0,-2.319709,2.621531,1.091329
9997,1.0,2.200028,-1.506163,-1.209778
9998,1.0,-2.118740,3.184799,0.465106


In [3]:
from carla.models.catalog import MLModelCatalog


training_params = {"lr": 0.01, "epochs": 10, "batch_size": 16, "hidden_size": [18, 9, 3]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)

balance on test set 0.5278666666666667, balance on test set 0.5092
Epoch 0/9
----------
train Loss: 0.3782 Acc: 0.8271

test Loss: 0.3704 Acc: 0.8308

Epoch 1/9
----------
train Loss: 0.3635 Acc: 0.8369

test Loss: 0.3613 Acc: 0.8364

Epoch 2/9
----------
train Loss: 0.3620 Acc: 0.8376

test Loss: 0.3683 Acc: 0.8352

Epoch 3/9
----------
train Loss: 0.3631 Acc: 0.8359

test Loss: 0.3672 Acc: 0.8340

Epoch 4/9
----------
train Loss: 0.3625 Acc: 0.8356

test Loss: 0.3731 Acc: 0.8328

Epoch 5/9
----------
train Loss: 0.3632 Acc: 0.8321

test Loss: 0.3641 Acc: 0.8332

Epoch 6/9
----------
train Loss: 0.3608 Acc: 0.8349

test Loss: 0.3700 Acc: 0.8356

Epoch 7/9
----------
train Loss: 0.3624 Acc: 0.8369

test Loss: 0.3628 Acc: 0.8348

Epoch 8/9
----------
train Loss: 0.3611 Acc: 0.8393

test Loss: 0.3662 Acc: 0.8376

Epoch 9/9
----------
train Loss: 0.3601 Acc: 0.8373

test Loss: 0.3663 Acc: 0.8320



In [4]:
from carla.models.negative_instances import predict_negative_instances
from carla.recourse_methods.catalog.causal_recourse import (
    CausalRecourse,
    constraints,
    samplers,
)


In [5]:
# get factuals
factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:5]

In [6]:
hyperparams = {
    "optimization_approach": "brute_force",
    "num_samples": 10,
    "scm": scm,
    "constraint_handle": constraints.point_constraint,
    "sampler_handle": samplers.sample_true_m0,
}
cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

Initialization Starting
Initialization Finished
Start get CF
index 1
1
2
3
4
5
node x1
node x2
node x3
Possible Actions [[-12.88409, -10.3755, -7.8669, -5.3583, -2.8497, -0.34111, 2.16749, 4.67609, 7.18469, 9.69328, None], [-12.74594, -9.79672, -6.8475, -3.89828, -0.94906, 2.00016, 4.94938, 7.8986, 10.84782, 13.79704, None], [-7.61488, -5.72381, -3.83274, -1.94166, -0.05059, 1.84049, 3.73156, 5.62263, 7.51371, 9.40478, None]]
tuples [(-12.88409, -12.74594, -7.61488), (-12.88409, -12.74594, -5.72381), (-12.88409, -12.74594, -3.83274), (-12.88409, -12.74594, -1.94166), (-12.88409, -12.74594, -0.05059), (-12.88409, -12.74594, 1.84049), (-12.88409, -12.74594, 3.73156), (-12.88409, -12.74594, 5.62263), (-12.88409, -12.74594, 7.51371), (-12.88409, -12.74594, 9.40478), (-12.88409, -12.74594, None), (-12.88409, -9.79672, -7.61488), (-12.88409, -9.79672, -5.72381), (-12.88409, -9.79672, -3.83274), (-12.88409, -9.79672, -1.94166), (-12.88409, -9.79672, -0.05059), (-12.88409, -9.79672, 1.84049), 

,x2,x1,x3
0,-3.048152,0.506218,0.242393
1,-1.812545,1.647384,-0.569361
2,-2.909084,3.653619,-1.089874
3,-3.716418,2.675338,-2.408550
4,7.977101,-3.973634,0.144281
